# DeepPavlov: Transfer Learning with BERT

Valerie Nayak <br>
December 3, 2019 <br><br>
This notebook implements the pretrained BERT model for question answering, which I plan on modifying after implementation.

## BERT input representation
Text preprocessing for BERT relies on tokenizing text on subtokens (or WordPieces). Then BERT internally represents each subtoken as sum of three vectors:
* subtoken embedding
* segment embedding
* position embedding

<img src="https://github.com/deepmipt/dp_tutorials/blob/master/img/BERT_input.png?raw=1" width="75%" />

## BERT for text classification
When we want to use BERT model for text classification task we can train only one dense layer on top of the output from the last BERT Transformer layer for special `[CLS]` token.

<img src="https://github.com/deepmipt/dp_tutorials/blob/master/img/BERT_classification.png?raw=1" width="75%" />

Install DeepPavlov library:

In [1]:
! pip install deeppavlov

## BERT for Question Answering (Stanford Question Answering Dataset)

One can use BERT model for extractive Question Answering, e.g.,
context:
```markdown
In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under gravity. The main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail… Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals **within a cloud**. Short, intense periods of rain in scattered locations are called “showers”.
```
and question:
```
Where do water droplets collide with ice crystals to form precipitation?
```
Answer is always a span from context.

To solve this task with BERT model all we need is to train two dense layes to predict answer start and answer end positions:

<img src="https://github.com/deepmipt/dp_tutorials/blob/master/img/BERT_QA.png?raw=1" width="50%" />

Downloading and interacting with pre-trained model:

In [4]:
! python -m deeppavlov install squad_bert

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-03 16:01:19.697 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'squad_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/squad/squad_bert.json'
     |████████████████████████████████| 109.2MB 45kB/s 
     |████████████████████████████████| 491kB 39.9MB/s 
     |████████████████████████████████| 3.2MB 30.8MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req

In [5]:
from deeppavlov import build_model, configs

model = build_model(configs.squad.squad_bert, download=True)

2019-12-03 16:03:11.248 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_bert.tar.gz download because of matching hashes
2019-12-03 16:03:14.867 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/cased_L-12_H-768_A-12.zip download because of matching hashes


Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2019-12-03 16:03:41.60 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_bert/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_bert/model


In [6]:
model(['In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under gravity. The main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail… Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals within a cloud. Short, intense periods of rain in scattered locations are called “showers”.'], 
      ['Where do water droplets collide with ice crystals to form precipitation?'])

[['within a cloud'], [305], [15922.36328125]]

Model returns an answer, position in characters and confidence.

To train model on your data you should put it json files in SQuAD format: https://rajpurkar.github.io/SQuAD-explorer/

These json files contain paragraphs, questions and answers.


In [8]:
! python run_squad.py \
  --vocab_file=$BERT_LARGE_DIR/vocab.txt \
  --bert_config_file=$BERT_LARGE_DIR/bert_config.json \
  --init_checkpoint=$BERT_LARGE_DIR/bert_model.ckpt \
  --do_train=True \
  --train_file=$SQUAD_DIR/train-v2.0.json \
  --do_predict=True \
  --predict_file=$SQUAD_DIR/dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://some_bucket/squad_large/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True

python3: can't open file 'run_squad.py': [Errno 2] No such file or directory
